# Helios ML Framework - Titanic Competition

This notebook demonstrates the Helios ML framework with:
- ISR-governed validation (T≥1.5)
- QMV-monitored consistency (C<0.03)
- RLAD feature engineering
- MoT ensemble voting
- Stratified k-fold CV
- Adversarial validation

**Target**: 78-82% test accuracy with full ISR/QMV audit trail

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
from helios import ISRValidator, QMVMonitor, FeatureEngineer, EnsembleOrchestrator
import warnings
warnings.filterwarnings('ignore')

print("Helios ML Framework initialized")

## 1. Load Data

Load Titanic dataset (assumes data is in ../data/ directory)

In [ ]:
# Load training data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

print(f"Training set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")
print(f"\nTraining data preview:")
train_df.head()

## 2. Feature Engineering with RLAD

Apply RLAD (Robust Learning with Adversarial Defense) feature engineering

In [ ]:
# Initialize feature engineer
feature_engineer = FeatureEngineer()

# Engineer features for training set
train_engineered = feature_engineer.engineer_features(train_df, is_training=True)
test_engineered = feature_engineer.engineer_features(test_df, is_training=False)

print(f"Engineered training set shape: {train_engineered.shape}")
print(f"\nEngineered features:")
print(train_engineered.columns.tolist())

## 3. Prepare Features for Training

In [ ]:
# Prepare features
X_train, y_train = feature_engineer.prepare_for_training(
    train_engineered,
    target_col='Survived',
    scale_features=True
)

print(f"Training features shape: {X_train.shape}")
print(f"Selected features: {X_train.columns.tolist()}")

## 4. Adversarial Validation

Check for distribution shift between train and test sets

In [ ]:
# Prepare test features
feature_cols = feature_engineer.select_features(test_engineered)
X_test = test_engineered[feature_cols].copy()

# Run adversarial validation
adv_results = feature_engineer.adversarial_validation(X_train, X_test, n_folds=5)

print("Adversarial Validation Results:")
print(f"  Mean AUC: {adv_results['mean_auc']:.4f}")
print(f"  Distribution Shift: {adv_results['distribution_shift']}")
print(f"  Interpretation: {adv_results['interpretation']}")
print(f"\nAUC < 0.6 indicates similar distributions (good)")
print(f"AUC > 0.75 indicates significant distribution shift (concerning)")

## 5. ISR Validation

Validate Information Stability Ratio (T ≥ 1.5)

In [ ]:
from sklearn.model_selection import train_test_split

# Split for ISR validation
X_train_isr, X_val_isr, y_train_isr, y_val_isr = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

# Initialize ISR validator
isr_validator = ISRValidator(threshold=1.5)

# Validate ISR
isr_metrics = isr_validator.validate(
    X_train_isr,
    X_val_isr,
    metadata={'stage': 'pre-training'}
)

print("ISR Validation Results:")
print(f"  ISR Value: {isr_metrics.isr_value:.4f}")
print(f"  Valid (T ≥ 1.5): {isr_metrics.is_valid}")
print(f"  Threshold: {isr_metrics.threshold}")

if isr_metrics.is_valid:
    print("\n✓ ISR validation PASSED - Data is stable for training")
else:
    print("\n✗ ISR validation FAILED - Data stability concerns detected")

## 6. Train Ensemble with Stratified K-Fold CV

Train MoT (Mixture of Techniques) ensemble with multiple models

In [ ]:
# Initialize ensemble orchestrator
ensemble = EnsembleOrchestrator(random_state=42)

# Train with stratified k-fold CV
print("Training ensemble with stratified 5-fold CV...\n")
cv_scores = ensemble.train_with_cv(
    X_train,
    y_train,
    n_folds=5,
    verbose=True
)

print("\nTraining completed!")

## 7. QMV Monitoring

Monitor Quality Metric Variance (C < 0.03)

In [ ]:
# Initialize QMV monitor
qmv_monitor = QMVMonitor(threshold=0.03)

# Validate QMV for each model
print("QMV Validation Results:\n")
for model_name, scores in cv_scores.items():
    qmv_metrics = qmv_monitor.validate(
        scores,
        metric_name=model_name,
        metadata={'stage': 'cross-validation'}
    )
    
    status = "✓ PASS" if qmv_metrics.is_valid else "✗ FAIL"
    print(f"{model_name}:")
    print(f"  QMV: {qmv_metrics.qmv_value:.6f}")
    print(f"  Status (C < 0.03): {status}")
    print()

## 8. Ensemble Performance Summary

In [ ]:
# Get CV summary
cv_summary = ensemble.get_cv_summary()

print("Cross-Validation Summary:")
print(cv_summary.to_string(index=False))

# Check if target accuracy is met
best_score = cv_summary['mean_score'].max()
print(f"\nBest mean CV accuracy: {best_score:.4f}")

if 0.78 <= best_score <= 0.82:
    print("✓ Target accuracy range (78-82%) ACHIEVED")
elif best_score > 0.82:
    print("⚠ Accuracy exceeds target range - potential overfitting")
else:
    print("✗ Accuracy below target range - model tuning needed")

## 9. Feature Importance Analysis

In [ ]:
# Get feature importance
importance = ensemble.get_feature_importance(method='mean')

if len(importance) > 0:
    print("Top 10 Most Important Features:")
    print(importance.head(10))
    
    # Visualize if matplotlib is available
    try:
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(10, 6))
        importance.head(10).plot(kind='barh')
        plt.xlabel('Mean Importance')
        plt.title('Top 10 Feature Importance (Averaged Across Models)')
        plt.tight_layout()
        plt.show()
    except ImportError:
        pass

## 10. Generate Predictions for Test Set

In [ ]:
# Make predictions on test set
test_predictions = ensemble.predict(X_test, voting='weighted')

# Create submission file
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': test_predictions
})

submission.to_csv('../data/submission.csv', index=False)
print("Predictions saved to ../data/submission.csv")
print(f"\nPrediction distribution:")
print(submission['Survived'].value_counts())

## 11. Export Audit Trails

Export ISR and QMV audit trails for compliance

In [ ]:
import os

# Create audit trail directory
os.makedirs('../audit_trails', exist_ok=True)

# Export ISR audit trail
isr_validator.export_audit_trail('../audit_trails/isr_audit_trail.csv')
print("ISR audit trail exported to ../audit_trails/isr_audit_trail.csv")

# Export QMV audit trail
qmv_monitor.export_audit_trail('../audit_trails/qmv_audit_trail.csv')
print("QMV audit trail exported to ../audit_trails/qmv_audit_trail.csv")

# Display ISR summary
print("\nISR Audit Trail Summary:")
print(isr_validator.get_audit_summary())

# Display QMV summary
print("\nQMV Audit Trail Summary:")
print(qmv_monitor.get_audit_summary())

## Summary

This notebook demonstrated the complete Helios ML framework:

1. ✓ RLAD feature engineering with robust transformations
2. ✓ Adversarial validation for distribution shift detection
3. ✓ ISR validation (T ≥ 1.5) for data stability
4. ✓ Stratified k-fold cross-validation
5. ✓ MoT ensemble with 5 diverse models
6. ✓ QMV monitoring (C < 0.03) for performance consistency
7. ✓ Full ISR/QMV audit trail generation
8. ✓ Target accuracy range: 78-82%

The framework ensures reproducible, auditable, and reliable ML predictions.